In [3]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization
from itertools import product
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import random
from collections import deque

In [4]:
# Load model
# train_data_folder = "/content/drive/MyDrive/PAM/test" 
model_path = './Model___Date_Time_2024_09_12__06_45_44___Loss_0.42576849460601807___Accuracy_0.9131770730018616.h5'

# 4. Load the best-performing model (if needed)
model = load_model(model_path)
print(f"Loaded best model from: {model_path}")

Loaded best model from: ./Model___Date_Time_2024_09_12__06_45_44___Loss_0.42576849460601807___Accuracy_0.9131770730018616.h5


Code for Predicting Activity from a Live Camera Feed:

In [21]:
# 1. Define a function to process frames from the webcam
def process_live_frames(img_size=(64, 64), sequence_length=30):
    cap = cv2.VideoCapture(0)  # Open webcam (0 is the default for most cameras)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Resize the frame to match the model input size
        resized_frame = cv2.resize(frame, img_size)
        frames.append(resized_frame)

        # Display the frame
        cv2.imshow('Live Video Feed', frame)

        # If we have enough frames for a sequence, yield them for prediction
        if len(frames) == sequence_length:
            yield np.array(frames)  # Return sequence as numpy array
            frames = []  # Reset for the next sequence

        # Press 'q' to exit the live feed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 2. Function to predict activity on live frames
def predict_live_activity(model, activity_classes, img_size=(64, 64), sequence_length=30):
    for frames in process_live_frames(img_size, sequence_length):
        frames = np.expand_dims(frames, axis=0)  # Add batch dimension
        prediction = model.predict(frames)
        predicted_label = np.argmax(prediction, axis=1)[0]
        predicted_activity = activity_classes[predicted_label]

        print(f"Predicted Activity: {predicted_activity}")

# 3. Load the pre-trained model and activity classes
# model_path = "path_to_saved_model.h5"  # Path to your saved model
model = load_model(model_path)

# Assuming `activity_classes` is the list of activities in your training data
activity_classes = ['Activity1', 'Activity2', 'Activity3']

# 4. Start the live prediction
predict_live_activity(model, activity_classes, img_size=(64, 64), sequence_length=30)


In [22]:
image_height, image_width = 64, 64
max_images_per_class = 8000

classes_list = ['Brought Back From Therapy (Transfer To Bed)',
                'Doctor Visit',
                'Nurse Visit',
                'Therapy',
                'EVS Visit',
                'Eating',
                'Lying In Bed',
                'Watching TV',
                'Asleep Trying to sleep',
                'Family',
                'Sitting In Wheelchair',
                'Talking on the Phone']

model_output_size = len(classes_list)

In [23]:
def make_average_predictions(predictions_frames_count):
    
    # Initializing the Numpy array which will store Prediction Probabilities
    predicted_labels_probabilities_np = np.zeros((predictions_frames_count, model_output_size), dtype = np.float)

    # Reading the Video File using the VideoCapture Object
    # video_reader = cv2.VideoCapture(video_file_path)
    # Open webcam
    video_reader = cv2.VideoCapture(0)

    # Getting The Total Frames present in the video 
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculating The Number of Frames to skip Before reading a frame
    skip_frames_window = video_frames_count // predictions_frames_count

    for frame_counter in range(predictions_frames_count): 

        # Setting Frame Position
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Reading The Frame
        _ , frame = video_reader.read() 

        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255

        # Passing the Image Normalized Frame to the model and receiving Predicted Probabilities.
        predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis = 0))[0]

        # Appending predicted label probabilities to the deque object
        predicted_labels_probabilities_np[frame_counter] = predicted_labels_probabilities

    # Calculating Average of Predicted Labels Probabilities Column Wise 
    predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)

    # Sorting the Averaged Predicted Labels Probabilities
    predicted_labels_probabilities_averaged_sorted_indexes = np.argsort(predicted_labels_probabilities_averaged)[::-1]

    # Iterating Over All Averaged Predicted Label Probabilities
    for predicted_label in predicted_labels_probabilities_averaged_sorted_indexes:

        # Accessing The Class Name using predicted label.
        predicted_class_name = classes_list[predicted_label]

        # Accessing The Averaged Probability using predicted label.
        predicted_probability = predicted_labels_probabilities_averaged[predicted_label]

        print(f"CLASS NAME: {predicted_class_name}   AVERAGED PROBABILITY: {(predicted_probability*100):.2}")
    
    # Closing the VideoCapture Object and releasing all resources held by it. 
    video_reader.release()

In [24]:
frames = 50
output_video_file_path = './predictions'
make_average_predictions(frames)

/var/folders/_9/4g32l3792w94_gk4_16tnt7c0000gn/T/ipykernel_40734/1513100821.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predicted_labels_probabilities_np = np.zeros((predictions_frames_count, model_output_size), dtype = np.float)


1/1 [==============================] - 0s 11ms/step
CLASS NAME: Nurse Visit   AVERAGED PROBABILITY: 1e+02
CLASS NAME: EVS Visit   AVERAGED PROBABILITY: 1.3e-10
CLASS NAME: Watching TV   AVERAGED PROBABILITY: 5.7e-11
CLASS NAME: Asleep Trying to sleep   AVERAGED PROBABILITY: 3e-11
CLASS NAME: Brought Back From Therapy (Transfer To Bed)   AVERAGED PROBABILITY: 2.8e-14
CLASS NAME: Talking on the Phone   AVERAGED PROBABILITY: 2.3e-19
CLASS NAME: Eating   AVERAGED PROBABILITY: 5e-21
CLASS NAME: Family   AVERAGED PROBABILITY: 4.2e-24
CLASS NAME: Sitting In Wheelchair   AVERAGED PROBABILITY: 1.8e-24
CLASS NAME: Lying In Bed   AVERAGED PROBABILITY: 3.3e-27
CLASS NAME: Doctor Visit   AVERAGED PROBABILITY: 6.8e-28
CLASS NAME: Therapy   AVERAGED PROBABILITY: 2.2e-31


In [27]:
def predict_on_live_video(output_file_path, window_size):
    # Initialize a Deque Object with a fixed size which will be used to implement moving/rolling average functionality.
    predicted_labels_probabilities_deque = deque(maxlen = window_size)

    # Open webcam
    video_reader = cv2.VideoCapture(1)

    # Getting the width and height of the video 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Writing the Overlayed Video Files Using the VideoWriter Object
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (original_video_width, original_video_height))

    while True: 

        # Reading The Frame
        status, frame = video_reader.read() 

        if not status:
            break

        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255

        # Passing the Image Normalized Frame to the model and receiving Predicted Probabilities.
        predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis = 0))[0]

        # Appending predicted label probabilities to the deque object
        predicted_labels_probabilities_deque.append(predicted_labels_probabilities)

        # Assuring that the Deque is completely filled before starting the averaging process
        if len(predicted_labels_probabilities_deque) == window_size:

            # Converting Predicted Labels Probabilities Deque into Numpy array
            predicted_labels_probabilities_np = np.array(predicted_labels_probabilities_deque)

            # Calculating Average of Predicted Labels Probabilities Column Wise 
            predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)

            # Converting the predicted probabilities into labels by returning the index of the maximum value.
            predicted_label = np.argmax(predicted_labels_probabilities_averaged)

            # Accessing The Class Name using predicted label.
            predicted_class_name = classes_list[predicted_label]
          
            # Overlaying Class Name Text Ontop of the Frame
            cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Writing The Frame
        video_writer.write(frame)


        cv2.imshow('Predicted Frames', frame)
        print('Predicted class name', predicted_class_name)

        # Press 'q' to exit the live feed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    
    cv2.destroyAllWindows()
    video_reader.release()
    
    # Closing the VideoCapture and VideoWriter objects and releasing all resources held by them. 
    
    # video_writer.release()

In [28]:
window_size = 1
output_video_file_path = './predictions'
predict_on_live_video(output_video_file_path, window_size)

1/1 [==============================] - 0s 14ms/step
Predicted class name Nurse Visit
1/1 [==============================] - 0s 12ms/step


[ WARN:0@761.677] global cap.cpp:645 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:430: error: (-215:Assertion failed) !filename_pattern.empty() in function 'open'




Predicted class name Nurse Visit
1/1 [==============================] - 0s 12ms/step
Predicted class name Nurse Visit
1/1 [==============================] - 0s 11ms/step
Predicted class name Nurse Visit
1/1 [==============================] - 0s 11ms/step
Predicted class name Nurse Visit
1/1 [==============================] - 0s 11ms/step
Predicted class name Nurse Visit
1/1 [==============================] - 0s 11ms/step
Predicted class name Nurse Visit
1/1 [==============================] - 0s 11ms/step
Predicted class name Nurse Visit
1/1 [==============================] - 0s 11ms/step
Predicted class name Nurse Visit
1/1 [==============================] - 0s 11ms/step
Predicted class name Nurse Visit
1/1 [==============================] - 0s 11ms/step
Predicted class name Nurse Visit
1/1 [==============================] - 0s 11ms/step
Predicted class name Nurse Visit
1/1 [==============================] - 0s 12ms/step
Predicted class name Nurse Visit
1/1 [===========================

: 